# Homework 3
In this homework assignment, you will implement a univariate feature selection method. 

You will be given a toy dataset called 'Car Evaluation Data Set' (see: http://archive.ics.uci.edu/ml/datasets/Car+Evaluation for details).
You are not required to, but advised to test your code with the toy dataset, or any other dataset that contains categorical variables.

The given dataset contains six descriptive features and a target variable. Each of those are ordinal scale, categorical variables. The name of the target feature is 'evaluation'. 

Note here that you are expected to write your own code, so DO NOT COPY AND PASTE CODE OR USE LIBRARY FUNCTIONS. The goal of the homework is not to see if you can call library functions but to have you practice with the impurity measures and feature selection techniques.


In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

### Read the dataset

In [34]:
edf = pd.read_csv('../Homework_3/careval.csv')
# edf.head()
edf.info()
edf.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
buying        1728 non-null object
maint         1728 non-null object
doors         1728 non-null object
persons       1728 non-null object
lug_boot      1728 non-null object
safety        1728 non-null object
evaluation    1728 non-null object
dtypes: object(7)
memory usage: 94.6+ KB


,buying,maint,doors,persons,lug_boot,safety,evaluation
count,1728,1728,1728,1728,1728,1728,1728
unique,4,4,4,3,3,3,4
top,high,high,5more,2,med,high,unacc
freq,432,432,432,576,576,576,1210


You will create a method called IUFS (impurity-based univariate feature selection), which will select the most informative features with a univariate feature selection schema. This feature selection method will take the dataset, name of the target variable, number of features to be selected (k) and the measure of impurity as an input, and will output the names of k best features based on the information gain. You are expected to implement information gain, entropy and Gini index functions. Note here that this will be a univariate selection, which means that you need to test the features individually.

In [52]:
# entropy (H)

def entropy(feature, dataset):
    """Calculates the entropy of a feature in a given dataset.
    
    Parameters
    ----------
    feature: str
        name of the feature
    dataset: pd.DataFrame
        dataframe for the dataset
    Returns
    -------
    float
        entropy for the feature in the dataset
    """
    ##your implementation goes here
#     P = dataset[feature].value_counts(normalize=True, sort=False)
    P = pd.value_counts(dataset[feature], normalize=True, sort=False)
#     print(P)
    return -(P * np.log2(P)).sum()
#     pass

print(entropy('buying', edf) )
print(entropy('doors', edf) )

2.0
2.0


In [51]:
# gini index (Gini)

def gini(feature, dataset):
    """Calculates the gini index of a feature in a given dataset.
    
    Parameters
    ----------
    feature: str
        name of the feature
    dataset: pd.DataFrame
        dataframe for the dataset
    Returns
    -------
    float
        gini index for the feature in the dataset
    """
    vc = dataset[feature].value_counts(normalize=True, sort=False)
    return 1.0 - np.power(vc, 2.0).sum()
    
    
gini('evaluation', edf) 
gini('buying', edf) 

0.6666666666666667

In [48]:
# information gain (IG)

def IG(feature, target, dataset, measure):
    """Calculates the information gain of a feature for a given target variable and a dataset.
    
    Parameters
    ----------
    feature: str
        name of the feature
    target: str
        name of the target variable
    dataset: pd.DataFrame
        dataframe for the dataset
    measure: str ('entropy' or 'gini')
        measure of impurity to be used
    Returns
    -------
    float
        information gain for the feature in the dataset for a given target variable
    """
    
    #calculate initial impurity
    impurity = -1
    if measure == 'gini':
        impurity = gini(target, dataset)
    elif measure == 'entropy':
        impurity = entropy(target, dataset)
    
    #split and calculate remaining impurity
    levels = dataset[feature].unique()
    remsum = 0.0
    for level in levels:
        li_df = dataset[dataset[feature]==level]
        remi = -1.0
        if measure == 'gini':
            remi = gini(target, li_df)
        elif measure == 'entropy':
            remi = entropy(target, li_df)
        remsum = remsum + (li_df.shape[0]/dataset.shape[0])*remi
    
    return impurity - remsum

print(IG('buying','evaluation', edf, 'entropy') )
print(IG('buying','evaluation', edf, 'gini') )

0.09644896916961399
0.014286077889231918


In [37]:
def IUFS(target, dataset, k, measure='entropy'):
    """Finds k most informative features in the given dataset based on the target variable
        using information gain with the selected measure.
        
    Parameters
    ----------
    target: str
        name of the target variable
    dataset: pd.DataFrame
        dataframe for the dataset
    k: int
        number of features to return, must be less than or equal to number of descriptive features in dataset.
        in other words, 0 < k < len(dataset.columns).
    measure: str, 'entropy' or 'gini'
        measure of impurity
    Returns
    -------
    list
        returns a list of k feature names, selected based on univariate selection schema
    """
    ranking = {}
    for feature in dataset.columns:
        if feature==target:
            continue
        IG_f = IG(feature, target, edf, measure)
        ranking[feature]=IG_f
    
    return sorted(ranking, key=ranking.get, reverse=True)[:k]

IUFS('evaluation', edf, 2, measure='entropy')

['safety', 'persons']

### Bonus
Improve the IUFS by including an option for gain ratio. Gain ratio is an alternative to information gain and can be used with either of the Gini index or entropy measures.  

In [46]:
def GR(feature, target, dataset, measure):
    """Calculates the gain ratio of a feature for a given target variable and a dataset.
    
    Parameters
    ----------
    feature: str
        name of the feature
    target: str
        name of the target variable
    dataset: pd.DataFrame
        dataframe for the dataset
    measure: str ('entropy' or 'gini')
        measure of impurity to be used
    Returns
    -------
    float
        gain ratio for the feature in the dataset for a given target variable
    """
    return IG(feature,target, dataset, measure) / entropy(feature, dataset)
    
print(GR('buying','evaluation', edf, 'entropy') )
print(GR('buying','evaluation', edf, 'gini') )

0.04822448458480699
0.007143038944615959


In [39]:
def IUFS2(target, dataset, k, measure='entropy', gain='IG'):
    """Finds k most informative features in the given dataset based on the target variable
        using information gain with the selected measure.
        
    Parameters
    ----------
    target: str
        name of the target variable
    dataset: pd.DataFrame
        dataframe for the dataset
    k: int
        number of features to return, must be less than or equal to number of descriptive features in dataset.
        in other words, 0 < k < len(dataset.columns).
    measure: str, 'entropy' or 'gini'
        measure of impurity
    gain: str, 'IG' or 'GR'
        feature selection metric ('IG' for information gain, 'GR' for gain ratio)
    Returns
    -------
    list
        returns a list of k feature names, selected based on univariate selection schema
    """
    ranking = {}
    for feature in dataset.columns:
        if feature==target:
            continue
        if gain == 'IG':
            ranking[feature] = IG(feature, target, edf, measure)
        elif gain == 'GR':
            ranking[feature] = GR(feature, target, edf, measure)
            
    return sorted(ranking, key=ranking.get, reverse=True)[:k]

IUFS2('evaluation', edf, 2, measure='gini', gain='GR')

['safety', 'persons']